# doc2vec – Paragraph Vectors

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import nltk.data

In [2]:
# Read data from files 
train = pd.read_csv( "labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )
test = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print "Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size )


Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [3]:
def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [4]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [11]:
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec

def review_to_doc(uid, review,  remove_stopwords=False ):
    # words and label as array
    return LabeledSentence(words=review_to_wordlist(review, remove_stopwords), tags=['SENT_%s' % uid])

# note: we might use sentences here instead of a whole review, right?
docs = []
i = 0
for review in train["review"]:
    docs.append(review_to_doc(i, review))
    i += 1

for review in unlabeled_train["review"]:
    docs.append(review_to_doc(i, review))
    i += 1
docs

[TaggedDocument(words=[u'with', u'all', u'this', u'stuff', u'going', u'down', u'at', u'the', u'moment', u'with', u'mj', u'i', u've', u'started', u'listening', u'to', u'his', u'music', u'watching', u'the', u'odd', u'documentary', u'here', u'and', u'there', u'watched', u'the', u'wiz', u'and', u'watched', u'moonwalker', u'again', u'maybe', u'i', u'just', u'want', u'to', u'get', u'a', u'certain', u'insight', u'into', u'this', u'guy', u'who', u'i', u'thought', u'was', u'really', u'cool', u'in', u'the', u'eighties', u'just', u'to', u'maybe', u'make', u'up', u'my', u'mind', u'whether', u'he', u'is', u'guilty', u'or', u'innocent', u'moonwalker', u'is', u'part', u'biography', u'part', u'feature', u'film', u'which', u'i', u'remember', u'going', u'to', u'see', u'at', u'the', u'cinema', u'when', u'it', u'was', u'originally', u'released', u'some', u'of', u'it', u'has', u'subtle', u'messages', u'about', u'mj', u's', u'feeling', u'towards', u'the', u'press', u'and', u'also', u'the', u'obvious', u'mes

In [12]:
import logging
from gensim.models import doc2vec

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel

# Initialize and train the model (this will take some time)
print "Training model..."
model = doc2vec.Doc2Vec(docs, workers=num_workers, \
            min_count = min_word_count)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "doc2vec_300features_40minwords_10context"
model.save(model_name)

Training model...


In [13]:
model.doesnt_match("man woman child kitchen".split())

'kitchen'

In [14]:
model.doesnt_match("france england germany berlin".split())

'berlin'